In [2]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model

In [3]:
semester_obj = Stream.objects.filter(slug='175157_1731147160').first()

In [13]:
stream_title = stream_obj.title
stream_code = stream_obj.stream_code
semester = 7
acedemic_year='2018-19'

In [14]:
subjects = PermanentSubject.objects.filter(degree=stream_title,stream_code=stream_code,acedemic_year=acedemic_year,sem_year=semester)

In [15]:
subjects.count()

14